In [ ]:
# analyze history
# get URLs with 300 words + 
# get tweets
# get model
# filter out non-domain specific news

In [ ]:
from proseflow.load import load

You can download your Browser History (up to 3 months) with this extension: https://chrome.google.com/webstore/detail/export-chrome-history/dihloblpkeiddiaojbagoecedbfpifdj?hl=en

or with python SQLite (https://cs.wellesley.edu/~srosassm/cs234/google/sql_chrome_srosassm.html#sec2)


In [ ]:
load("data/browserhistory_012021.csv")

No Match found in type inference, returning None
None shape {}


'None found'

In [ ]:
import pandas as pd
history = pd.read_csv("data/browserhistory_012021.csv")

In [ ]:
h = history
len(h)

56711

In [ ]:
h

,order,id,date,time,title,url,visitCount,typedCount,transition
0,0,69921,1/3/2021,18:24:31,Export Chrome History - Chrome Web Store,https://chrome.google.com/webstore/detail/expo...,1,0,link
1,1,69920,1/3/2021,18:24:16,Export Chrome History - Chrome Web Store,https://chrome.google.com/webstore/detail/expo...,1,1,typed
2,2,66876,1/3/2021,18:04:33,Simulacrum | Jitsi Meet,https://meet.jit.si/simulacrum,3,2,typed
3,3,59477,1/3/2021,18:02:36,Meet - The Little Typer (5),https://meet.google.com/nhi-xdzs-gbx?authuser=0,10,0,link
4,4,69919,1/3/2021,18:02:35,Meet - nhi-xdzs-gbx,https://meet.google.com/nhi-xdzs-gbx?authuser=...,2,0,link
...,...,...,...,...,...,...,...,...,...
56706,56706,30533,10/5/2020,19:31:50,"Google Calendar - Sunday, January 3, 2021, today",https://calendar.google.com/calendar/u/0/r?tab=mc,247,1,link
56707,56707,30533,10/5/2020,19:31:49,"Google Calendar - Sunday, January 3, 2021, today",https://calendar.google.com/calendar/u/0/r?tab=mc,247,1,typed
56708,56708,28193,10/5/2020,19:31:41,Slack | Sneha (LD15) | EF Europe15,https://app.slack.com/client/T018XV4B7AB/D019Y...,11,0,link
56709,56709,38351,10/5/2020,19:31:41,"Slack | Spencer, EF Team | EF Europe15",https://app.slack.com/client/T018XV4B7AB/D01BU...,6,0,link


In [ ]:
stop_urls = ["chrome.google", "github.com", "facebook.com", "roamresearch.com", ".pdf", "amazon.", "airbnb.", "/video", "linkedin.", "libgen.", "google.com", "file:///", "google.com", "duckduckgo.com", "twitter.com", "jitsi.", "slack.com", "youtube.com", "reddit.com", "meet.", "/accounts", "/billing"  ]
# cleaned = [row for index, row in h.iterrows() if not any([url in row["url"] for url in stop_urls])]
# len(cleaned)

import numpy as np
mask = [np.sum([u in v for u in stop_urls]) == 0 for v in h.url]
len(h[mask])

c = h[mask]
is_homepage_mask = [True if len([_ for _ in url.split("/") if _]) > 2 else False for url in c.url ]

cleaned = c[is_homepage_mask]
len(cleaned)

14629

In [ ]:
cleaned = cleaned.drop_duplicates(subset=["url"])
len(cleaned)

7554

In [ ]:
def get_post_text(url):
    return " ".join(bs_to_paragraphs(url_to_bs(url), "p", {}))

In [ ]:
def is_post(url):
    """an incredibly complex classifier. State of the Art"""
    try:
#       t = url_to_text(url)
        t = get_post_text(url)
        if len(t.split(" ")) > 400:
            print("article here at ", url)
            return True
    except:
        pass
    return False

In [ ]:
is_post('https://www.ldeming.com/')

False

In [ ]:
is_post_mask = [is_post(url) for url in cleaned.url[2000:5000]]
post_df = cleaned[2000:5000][is_post_mask]
post_df


article here at  https://www.nature.com/articles/s41467-020-19092-2
article here at  https://reiinakano.com/arbitrary-image-stylization-tfjs/
article here at  https://breandan.net/2020/06/30/graph-computation/
article here at  https://www.unisonweb.org/docs/codebase-organization
article here at  https://pchiusano.github.io/2019-10-24/musicians-and-programmers.html
article here at  https://en.wikipedia.org/wiki/Metaverse
article here at  https://en.wikipedia.org/wiki/High_Fidelity_(company)
article here at  https://en.wikipedia.org/wiki/High_Fidelity_Inc
article here at  https://en.wikipedia.org/wiki/NeosVR
article here at  https://en.wikipedia.org/wiki/Internet_metaphors
article here at  https://www.si.com/gambling/2020/10/13/what-is-the-metaverse
article here at  https://www.economist.com/technology-quarterly/2020/10/01/the-metaverse-is-coming
article here at  https://pchiusano.github.io/2013-05-22/future-of-software.html
article here at  http://rishi.is/posts/2020-08-12-the-problem.h

In [ ]:
post_df.to_csv ("data/parsed2", index = False, header=True)
#post_df.to_csv(index=False)


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import requests
from toolz import thread_first, thread_last
from typing import List

STRING_FUNCS = ["capitalize", "join", "count", "isalnum", "isalpha", "isascii", "isdecimal", "isdigit", "isidentifier", "islower", "isnumeric", "isspace", "istitle", "isupper", "lower", "lstrip", "replace", "rstrip", "split", "strip", "upper"]

for s_func in STRING_FUNCS: #PYTHON MAGIC
    exec("%s=getattr(str, s_func)" %s_func)
    
def pipe(*funcs:List[callable], thread="first"):
  thread = thread_first if thread == "first" else thread_last
  return lambda data: thread(data, *funcs)

clean_sentence = pipe(strip,
                     (replace, "\n", " ")
                     )

def bs_to_paragraphs(soup, tag, tag_filter, strip=True):
  """ Extracts all the text from the given tags and returns them as a list. """
  return [paragraph.get_text(strip=strip) for paragraph in soup.findAll(tag, tag_filter)]

def url_to_html(url: str):
  """ Takes a url and returns a request.Response object """
  return requests.get(url, headers = {'User-Agent': 'test'})


def url_to_base_url(url):
  """ Gets the base url from url (useful for pagination). """
  return "/".join(url.split("/")[:3])

def html_to_bs(html, parser="lxml"):
  """ Takes a request.Response object and returns a Beautiful Soup object"""
  return BeautifulSoup(html.content, parser)

def bs_replace(soup, source, target):
  """ Replaces source tag with target tag in Beautiful Soup object """
  for tag in soup.findAll(source):
    tag.replace_with(target)
  return soup

def bs_to_text(soup, strip=True):
  """ Extracts all the text from a Beautiful soup object. """
  return soup.get_text(strip=strip)

def bs_to_paragraphs(soup, tag, tag_filter, strip=True):
  """ Extracts all the text from the given tags and returns them as a list. """
  return [paragraph.get_text(strip=strip) for paragraph in soup.findAll(tag, tag_filter)]

def exception_handler(func):
    def inner_function(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except TypeError:
            print(f"{func.__name__} only takes numbers as the argument")
    return inner_function    

def bs_to_paragraphs(soup, tag, tag_filter, strip=True):
  """ Extracts all the text from the given tags and returns them as a list. """
  return [paragraph.get_text(strip=strip) for paragraph in soup.findAll(tag, tag_filter)]

url_to_text = pipe(url_to_html,
                      html_to_bs,
                      (bs_replace, "br", "\n"),
                      bs_to_text
                     )

url_to_bs = pipe(url_to_html, 
                 html_to_bs)


In [ ]:
len(get_post_text("https://alexdanco.com/2020/09/17/are-founders-allowed-to-lie/").split(" "))

2502

In [ ]:
#get_post_text('https://www.ldeming.com/longevityfaq')